In [1]:
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
import seaborn as sns
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
df= pd.read_csv("Dataset_de_200_Comunidades_para_Analise_Ambiental2.csv")

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Converter colunas categóricas para numéricas
df['Presença de Escola'] = df['Presença de Escola (Sim/Não)'].map({'Sim': 1, 'Não': 0})
df['Presença de Saúde'] = df['Presença de Unidade de Saúde (Sim/Não)'].map({'Sim': 1, 'Não': 0})

# Selecionar features para clusterização (incluindo coordenadas)
features = [
    'Latitude', 
    'Longitude',
    'Índice de Desmatamento (%)',
    'Acesso à Água Potável (%)',
    'Cobertura Vegetal (%)',
    'Renda Média Mensal (R$)',
    'Densidade Populacional (hab/km²)',
    'Frequência de Queimadas (ano)',
    'Distância de Área Urbana (km)',
    'Presença de Escola',
    'Presença de Saúde'
]

# Normalizar os dados
scaler = StandardScaler()
df_normalizado = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

In [3]:
# Peso 0.3 para coordenadas (valor entre 0 e 1, ajuste conforme necessidade)
df_normalizado['Latitude'] *= 0.3
df_normalizado['Longitude'] *= 0.3

In [4]:
from sklearn.cluster import KMeans

# Definir número de clusters (use o método do cotovelo para escolher K)
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(df_normalizado)

# Verificar distribuição dos clusters
print(df['Cluster'].value_counts())

Cluster
0    98
2    57
1    45
Name: count, dtype: int64


In [5]:
# Agrupar por cluster e calcular médias
cluster_stats = df.groupby('Cluster')[features].mean().reset_index()

# Ordenar clusters por vulnerabilidade (ex.: alto desmatamento + baixa renda)
cluster_stats['Score_Vulnerabilidade'] = (
    cluster_stats['Índice de Desmatamento (%)'] * 0.4 +
    (100 - cluster_stats['Acesso à Água Potável (%)']) * 0.3 +
    cluster_stats['Frequência de Queimadas (ano)'] * 0.3
)

cluster_stats = cluster_stats.sort_values('Score_Vulnerabilidade', ascending=False)
print(cluster_stats)

   Cluster  Latitude  Longitude  Índice de Desmatamento (%)  \
1        1 -3.057630 -49.700932                   47.995556   
2        2 -2.883245 -49.484008                   53.712281   
0        0 -2.976672 -49.610253                   43.838776   

   Acesso à Água Potável (%)  Cobertura Vegetal (%)  Renda Média Mensal (R$)  \
1                  61.562222              48.133333              1593.222222   
2                  71.029825              71.768421              1522.385965   
0                  68.536735              37.092857              1677.908163   

   Densidade Populacional (hab/km²)  Frequência de Queimadas (ano)  \
1                         85.111111                       5.911111   
2                        113.775439                       6.175439   
0                         63.047959                       5.265306   

   Distância de Área Urbana (km)  Presença de Escola  Presença de Saúde  \
1                      47.931111                 0.0           0.48888

In [6]:
import folium

# Cores para cada cluster (vermelho = mais vulnerável)
cores = {0: 'green', 1: 'orange', 2: 'red'}

# Criar mapa centrado no Brasil
mapa = folium.Map(location=[-15.79, -47.88], zoom_start=4)

# Adicionar marcadores
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color=cores[row['Cluster']],
        fill=True,
        popup=f"""
        <b>{row['Comunidade']}</b><br>
        Cluster: {row['Cluster']}<br>
        Desmatamento: {row['Índice de Desmatamento (%)']}%<br>
        Renda: R${row['Renda Média Mensal (R$)']:,.2f}
        """
    ).add_to(mapa)

# Salvar mapa interativo
mapa